## Measure the Energy Intensity of Running Tabular DDPMs ##

Things we care about:
1) Model Size
2) Model Type (CNN or MLP)
3) Dataset size
4) Batch Size
5) Data Type (Image or Tabular)
6) Training time
7) Sampling time
8) Sampled Data size

We hope to use the experiment run with this script to evaluate the carbon intensity of DDPMs

Training: Run DDPM training until convergence

Sampling: Generate 8192 samples with batch size parameter

In [1]:
#import the goods

import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import ImgDiffusionTools as img_ddpm
import time
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


from PIL import Image

"""
start = time.time()

-Run

end = time.time()

clock = end - start
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#Set up the loop:


machine_id = 'Quadro_RTX_4000_Shannon'
DS_sizes = [2048,4096]
batch_size = 64

timesteps = 100
patience = 100

#Net_sizes = [1,2,4,8]

Net_sizes = [1,2]

t_dim = 32


# Load in image data:
Wave_64 = []
for i in tqdm(range(0,2048)):
    img = Image.open('./dataset/Wave_4_Img_64/img_{}.png'.format(i))
    a = np.array(img)
    Wave_64.append([a])
    img = Image.open('./dataset/Wave_8_Img_64/img_{}.png'.format(i))
    a = np.array(img)
    Wave_64.append([a])
Wave_64 = (np.array(Wave_64).astype(np.float32) - 0)/255 * 2.0 - 1.0 # normalize to [-1,1]
print(Wave_64.shape) # should be (8192,1,256,256)

Wave_128= []
for i in tqdm(range(0,2048)):
    img = Image.open('./dataset/Wave_4_Img_128/img_{}.png'.format(i))
    a = np.array(img)
    Wave_128.append([a])
    img = Image.open('./dataset/Wave_8_Img_128/img_{}.png'.format(i))
    a = np.array(img)
    Wave_128.append([a])
Wave_128= (np.array(Wave_128).astype(np.float32) - 0)/255 * 2.0 - 1.0 # normalize to [-1,1]
print(Wave_128.shape) # should be (4096,1,128,128)


Wave = [Wave_64, Wave_128]







100%|██████████| 2048/2048 [00:00<00:00, 8565.60it/s]


(4096, 1, 64, 64)


100%|██████████| 2048/2048 [00:00<00:00, 3813.99it/s]


(4096, 1, 128, 128)


In [3]:
labels = ['run_id', 'model size', 'training epochs', 'train time', 'sample time 8192 samples']
Results = []
Results.append(labels)


for i in range(len(Wave)):
    for j in  range(len(DS_sizes)):
     # Set up the training data:
        DS = Wave[i][0:DS_sizes[j]]
        img_size = int(DS.shape[2])
        

        train_loader = DataLoader(DS, batch_size=batch_size, shuffle=True)


        for k in range(len(Net_sizes)):
            torch.cuda.empty_cache()
            I_model = img_ddpm.Img_DDPM_Env(img_size, Net_sizes[k], t_dim,timesteps, device)
            #print(torch.cuda.memory_summary())
            model_size = 0

            for p in I_model.model.parameters():
                model_size += p.numel()

            print('Model size: ', model_size)



            # Train the model
            start = time.time()
        
            epochs = I_model.train_loop_patience(patience, train_loader)

            end = time.time()
            clock_train = end - start

            # Sample the model
            start = time.time()

            X_gen = I_model.sample_images(64)

            
            end = time.time()

            clock_sample = end - start

            id = 'Img_' + str(img_size) + '_DS_' + str(DS_sizes[j]) + '_Net_' + str(k) 
            np.save('./generated_samples/Shannon_RTX_4000/' + id + '_Samples_' + machine_id + '.npy', X_gen)

            # Save a few images:
            for l in range(0,32):
                a = X_gen[l,0]
                img = Image.fromarray(a)
                path = './generated_samples/Shannon_RTX_4000/sample_images/'
                img.save(path+ id + '_Sample_' + str(l) + '_' + machine_id + '.png')

            Results.append([id,model_size, epochs, clock_train, clock_sample])



Model size:  4477
Epoch: 0 Loss: 0.7094660997390747
Epoch: 500 Loss: 0.15972065925598145
Epoch: 1000 Loss: 0.15973961353302002
Epoch: 1500 Loss: 0.16538196802139282
Epoch: 2000 Loss: 0.16020259261131287


100%|██████████| 100/100 [00:00<00:00, 2262.86it/s]


Model size:  13625
Epoch: 0 Loss: 0.5631052851676941
Epoch: 500 Loss: 0.07054269313812256


KeyboardInterrupt: 

In [ ]:
import csv

path = './Img_Wave_Results_' + machine_id + '.csv'

f =  open(path, 'w', newline='')
writer = csv.writer(f)
writer.writerows(Results)
f.close()